# IST 597 — Activity 01

We are using [Instructor](https://python.useinstructor.com/) for structured output
from LLMs.

In [4]:
import os
import instructor
from pydantic import BaseModel
from openai import OpenAI
import getpass

In [13]:
# You should NEVER share your API keys publicly.

# In this case, we are going to save it to a variable


_API_KEY = getpass.getpass("Your API key: \n")

Your API key: 
 ········


In [52]:
# We will store the API key so that we can access it
# throughout other activities
%store _API_KEY

Stored '_API_KEY' (str)


In [3]:
open_ai = OpenAI(api_key=_API_KEY)
client = instructor.from_openai(open_ai, mode=instructor.Mode.TOOLS_STRICT)

In [14]:
class Response(BaseModel):
    message: str

In [56]:
 def get_response(role, query):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Response,
        messages=[
            {
                "role": "user",
                "content": f"{role} {query}",
            },
        ],
    )

In [20]:
role = "You are a great poet"
query = "Write a haiku about IST 597 (HCAI): a class focusing on human-centered AI"

r = get_response(role, query)

In [21]:
print(r.message)

Human and machine,
Together weaving insights,
Hearts code, minds embrace.


**What are the options in `client.chat.completions.create`?**

Described in the [API documentation](https://platform.openai.com/docs/guides/text-generation?lang=python). Take a look at different roles:
* user
* developer
* assistant

In [54]:
r._raw_response

ChatCompletion(id='chatcmpl-AsyvAFZUH2FRm8ologTUfK8e2rlrb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fC0gnVdp6NU11CpdN7GUdrql', function=Function(arguments='{"message":"Human minds align,  \\nCrafting AI with pure heart,  \\nTogether we rise."}', name='Response'), type='function')]))], created=1737667256, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=26, prompt_tokens=72, total_tokens=98, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=None, prompt_token_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

**These values represent the raw output returned by the API**

You can see the object model described in the [documentation](https://platform.openai.com/docs/api-reference/chat/object).

## Different roles

Let's explore `user` and `developer` [roles](https://platform.openai.com/docs/guides/text-generation?lang=python).


In [27]:
class Email(BaseModel):
    subject: str
    message: str

In [59]:
def generate_email(developer, subject, to, sender, tone):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "developer",
                "content": developer,
            },
            {
                "role": "user",
                "content": f"""
                Write an email about {subject} to {to} from {sender}.
                The email should be {tone}.
                """,
            }
        ],
        response_model=Email,
    )

In [60]:
r = generate_email(developer="Respond in English",
               subject="Request for extending the assignment deadline",
               to="professor",
               sender="student",
               tone="formal")

In [61]:
print(r.subject)

Request for Extension of Assignment Deadline


In [62]:
print(r.message)

Dear Professor [Last Name],

I hope this message finds you well. I am writing to formally request an extension on the upcoming assignment due on [original deadline date]. 

Due to [brief explanation of your reasons, e.g., unforeseen personal circumstances, illness, or workload], I am concerned that I will not be able to complete the assignment to the best of my ability within the original timeframe. I want to ensure that I can submit work that reflects my true potential.

I would greatly appreciate it if you could grant me an extension until [proposed new deadline]. I believe that this additional time will allow me to deliver a more comprehensive and thoughtful assignment.

Thank you very much for considering my request. I am looking forward to your understanding and support.

Sincerely,

[Your Full Name]
[Your Student ID]
[Your Course Name]
[Your Contact Information]


In [63]:
r = generate_email(developer="Respond in Spanish",
               subject="Request for extending the assignment deadline",
               to="professor",
               sender="student",
               tone="formal")

In [64]:
print(r.message)

Estimado Profesor [Apellido del Profesor],

Espero que se encuentre bien. Me dirijo a usted para solicitar una extensión en el plazo de entrega de la tarea asignada para [nombre de la asignatura], cuya fecha límite es el [fecha de entrega original]. 

Lamentablemente, debido a [breve explicación de la razón, como problemas personales, enfermedades, carga de trabajo excesiva, etc.], me he visto en la necesidad de solicitar más tiempo para completar la tarea de manera adecuada. Estoy comprometido con mis estudios y quiero entregar un trabajo que refleje mi esfuerzo y dedicación. 

Agradecería mucho si pudiera considerar mi solicitud y permitirme un plazo adicional hasta [nueva fecha sugerida]. Estoy dispuesto a discutir esto en persona si lo considera necesario.

Agradezco de antemano su comprensión y apoyo.

Atentamente,

[Tu Nombre]  
[Tu Número de Estudiante]  
[Tu Curso]  
[Tu Correo Electrónico]
